In [1]:
import os
import re

import datetime

import pandas as pd
from dask import dataframe as dd
import zipfile
import plotly.express as px

from IPython.display import clear_output

In [2]:
#df = pd.read_csv("data/comed_month/comed_201901.csv", parse_dates=['date_time'])
#df[df.date_time.dt.day == 30].to_csv('data/comed_month/comed_201901_30.csv', index=False)

In [2]:
#path_data = "D:/CSUN/COMED/Comed DATA/201801.zip" 
path_data = "I:/CSUN_data/ENERGY DATA_Juan & Hevar/ComEd Anonymous Data 11.2019-08.2021/202005-202108" 
#zip = zipfile.ZipFile(path_data)
#files= (zip.namelist())
#month = '201711'
#zips = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(f'data/comed/{month}/') for f in filenames]
zips = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(path_data) for f in filenames]
zips = zips
N_zips = len(zips)

In [2]:
def get_HR(x):
    time_str = re.search(r'INTERVAL_HR(.*?)_ENERGY_QTY', x)[1]
    return time_str

selected_columns = ['ZIP_CODE', 'INTERVAL_READING_DATE', 
                    'DELIVERY_SERVICE_NAME', 'ACCOUNT_IDENTIFIER', 
                    'INTERVAL_HR0030_ENERGY_QTY', 
                    'INTERVAL_HR0100_ENERGY_QTY', 'INTERVAL_HR0130_ENERGY_QTY',
                    'INTERVAL_HR0200_ENERGY_QTY', 'INTERVAL_HR0230_ENERGY_QTY',
                    'INTERVAL_HR0300_ENERGY_QTY', 'INTERVAL_HR0330_ENERGY_QTY',
                    'INTERVAL_HR0400_ENERGY_QTY', 'INTERVAL_HR0430_ENERGY_QTY',
                    'INTERVAL_HR0500_ENERGY_QTY', 'INTERVAL_HR0530_ENERGY_QTY',
                    'INTERVAL_HR0600_ENERGY_QTY', 'INTERVAL_HR0630_ENERGY_QTY',
                    'INTERVAL_HR0700_ENERGY_QTY', 'INTERVAL_HR0730_ENERGY_QTY',
                    'INTERVAL_HR0800_ENERGY_QTY', 'INTERVAL_HR0830_ENERGY_QTY',
                    'INTERVAL_HR0900_ENERGY_QTY', 'INTERVAL_HR0930_ENERGY_QTY',
                    'INTERVAL_HR1000_ENERGY_QTY', 'INTERVAL_HR1030_ENERGY_QTY',
                    'INTERVAL_HR1100_ENERGY_QTY', 'INTERVAL_HR1130_ENERGY_QTY',
                    'INTERVAL_HR1200_ENERGY_QTY', 'INTERVAL_HR1230_ENERGY_QTY',
                    'INTERVAL_HR1300_ENERGY_QTY', 'INTERVAL_HR1330_ENERGY_QTY',
                    'INTERVAL_HR1400_ENERGY_QTY', 'INTERVAL_HR1430_ENERGY_QTY',
                    'INTERVAL_HR1500_ENERGY_QTY', 'INTERVAL_HR1530_ENERGY_QTY',
                    'INTERVAL_HR1600_ENERGY_QTY', 'INTERVAL_HR1630_ENERGY_QTY',
                    'INTERVAL_HR1700_ENERGY_QTY', 'INTERVAL_HR1730_ENERGY_QTY',
                    'INTERVAL_HR1800_ENERGY_QTY', 'INTERVAL_HR1830_ENERGY_QTY',
                    'INTERVAL_HR1900_ENERGY_QTY', 'INTERVAL_HR1930_ENERGY_QTY',
                    'INTERVAL_HR2000_ENERGY_QTY', 'INTERVAL_HR2030_ENERGY_QTY',
                    'INTERVAL_HR2100_ENERGY_QTY', 'INTERVAL_HR2130_ENERGY_QTY',
                    'INTERVAL_HR2200_ENERGY_QTY', 'INTERVAL_HR2230_ENERGY_QTY',
                    'INTERVAL_HR2300_ENERGY_QTY', 'INTERVAL_HR2330_ENERGY_QTY',
                    'INTERVAL_HR0000_ENERGY_QTY']

In [3]:
dict_col_agg = {}

for col in selected_columns[3:]:
    dict_col_agg[col] = 'sum'
#dict_col_agg['DELIVERY_SERVICE_NAME'] = lambda x: (x.value_counts().index, x.value_counts())
#dict_col_agg['DELIVERY_SERVICE_CLASS'] = lambda x: (x.value_counts().index, x.value_counts())
dict_col_agg['ACCOUNT_IDENTIFIER'] = 'count'

In [9]:
months = os.listdir(path_data)
months

['202001',
 '202002',
 '202004',
 '202005',
 '202006',
 '202007',
 '202008',
 '202009',
 '202010',
 '202011',
 '202012',
 '202101',
 '202102',
 '202103',
 '202104',
 '202105',
 '202106',
 '202107',
 '202108']

In [10]:
months = os.listdir(path_data)

day = 29

for month in months[:1]:
    zips = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(f'{path_data}/{month}/') for f in filenames if f.endswith('.zip')]
    zips = zips
    N_zips = len(zips)

    df_total = pd.DataFrame()
    #df_total = dd.from_pandas(df_total, npartitions=1)

    count = 0

    for zip_path in zips:
        if 'LIST' not in zip_path:
            zip = zipfile.ZipFile(zip_path)
            count = count + 1
            clear_output(wait=True)
            print(f'{count}/{N_zips} --------> {round(100*count/N_zips, 3)} % progress in month {month} and file {zip_path}')
            #for file in zip.namelist():
            #zip2 = zipfile.ZipFile(f'data/comed/{month}/{file}')
            #df = pd.read_csv(zip2.open(zip2.namelist()[0]))
            df = pd.read_csv(zip.open(zip.namelist()[0]), on_bad_lines=lambda x: x[:2] + x[2+1 :], engine='python', parse_dates=['INTERVAL_READING_DATE'])
            df = df[df.INTERVAL_READING_DATE.dt.day==day]
            df['ZIP_CODE'] = df['ZIP_CODE'].astype(str)
            df['ZIP_CODE'] = df['ZIP_CODE'].str[:5]
            df = df.rename(columns={'INTERVAL_HR2400_ENERGY_QTY': 'INTERVAL_HR0000_ENERGY_QTY'})
            #THISdf = df.groupby(['ZIP_CODE','INTERVAL_READING_DATE', 'DELIVERY_SERVICE_NAME']).agg(dict_col_agg).reset_index()
            df_total = pd.concat([df_total,df])
            #THISdf_total = df_total.groupby(['ZIP_CODE', 'INTERVAL_READING_DATE', 'DELIVERY_SERVICE_NAME']).sum().reset_index()

    df = df_total

    #df['DELIVERY_SERVICE_NAME'] = df.DELIVERY_SERVICE_NAME.astype(str)
    #df['DELIVERY_SERVICE_CLASS'] = df.DELIVERY_SERVICE_CLASS.astype(str)

    df = df.rename(columns={'INTERVAL_HR2400_ENERGY_QTY': 'INTERVAL_HR0000_ENERGY_QTY'})

    #df = df.drop(['DELIVERY_SERVICE_NAME', 'DELIVERY_SERVICE_CLASS'], axis=1)

    print('Reshaping data...')

    df = df[selected_columns].melt(
        id_vars=["ZIP_CODE", "ACCOUNT_IDENTIFIER", "INTERVAL_READING_DATE", 'DELIVERY_SERVICE_NAME'],
        var_name="Time",
        value_name="energy")


    df['Time'] = df.Time.map(lambda x: get_HR(x))
    #df['date_time'] = df.INTERVAL_READING_DATE.astype(str)  + " " + df.Time
    #df['date_time'] = df.date_time.map(lambda x: datetime.datetime.strptime(x,'%m/%d/%Y %H%M')).astype(str)
    df['date_time'] = df.Time.map(lambda x: datetime.datetime.strptime(f'01/{day}/2020 {x}','%m/%d/%Y %H%M')).astype(str)

    df = df[['ZIP_CODE', 'ACCOUNT_IDENTIFIER', 'DELIVERY_SERVICE_NAME', 'date_time', 'energy']]

    df['zip5'] = df.ZIP_CODE.str[:5]

    df = df.rename(columns={'ZIP_CODE': 'zip9','ACCOUNT_IDENTIFIER': 'n_acc', 'DELIVERY_SERVICE_NAME': 'service_name'})


    df = df[['zip5', 'n_acc', 'service_name', 'date_time', 'energy']]    
     

    print('Exporting data...')

    df.to_csv(f'../data/comed_month/comed_{month}_{day}_only.csv', index=False)

    del df
    del df_total



485/486 --------> 99.794 % progress in month 202001 and file I:/CSUN_data/ENERGY DATA_Juan & Hevar/ComEd Anonymous Data 11.2019-08.2021/202005-202108/202001/ANONYMOUS_DATA_202001_61769.csv.zip
Reshaping data...


MemoryError: Unable to allocate 4.26 GiB for an array with shape (3, 190510176) and data type object

In [4]:
df = pd.read_parquet('../data/comed_day/2020_02_19_temp.parquet')

In [5]:
df = df[selected_columns].melt(
    id_vars=["ZIP_CODE", "ACCOUNT_IDENTIFIER", "INTERVAL_READING_DATE", 'DELIVERY_SERVICE_NAME'],
    var_name="Time",
    value_name="energy")


df['Time'] = df.Time.map(lambda x: get_HR(x))
#df['date_time'] = df.INTERVAL_READING_DATE.astype(str)  + " " + df.Time
#df['date_time'] = df.date_time.map(lambda x: datetime.datetime.strptime(x,'%m/%d/%Y %H%M')).astype(str)

In [9]:
df.to_parquet('../data/comed_day/2020_01_29_melted.parquet', engine='pyarrow')

J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [2]:
df = dd.read_parquet('../data/comed_day/2020_01_29_melted.parquet')
df = df.repartition(npartitions=50)

In [3]:

#df['date_time'] = df.Time.map(lambda x: datetime.datetime.strptime(f'01/{29}/2020 {x}','%m/%d/%Y %H%M')).astype(str)

del df['INTERVAL_READING_DATE']

df['date_time'] = "01/29/2020 " + df.Time

df = df[['ZIP_CODE', 'ACCOUNT_IDENTIFIER', 'DELIVERY_SERVICE_NAME', 'date_time', 'energy']]

df['zip5'] = df.ZIP_CODE.str[:5]

df = df.rename(columns={'ZIP_CODE': 'zip9','ACCOUNT_IDENTIFIER': 'n_acc', 'DELIVERY_SERVICE_NAME': 'service_name'})


df = df[['zip5', 'n_acc', 'service_name', 'date_time', 'energy']]    
 

In [4]:
df.to_parquet('../data/comed_day/2020_01_29.parquet', engine='pyarrow')

J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:456: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:456: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:456: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_spars

In [44]:
#df['date_time'] = df.INTERVAL_READING_DATE.astype(str)  + " " + df.Time
df['date_time'] = df.Time.map(lambda x: datetime.datetime.strptime(f'02/{day}/2020 {x}','%m/%d/%Y %H%M')).astype(str)

df = df[['ZIP_CODE', 'ACCOUNT_IDENTIFIER', 'DELIVERY_SERVICE_NAME', 'date_time', 'energy']]

#df['zip5'] = df.ZIP_CODE.str[:5]

df = df.rename(columns={'ZIP_CODE': 'zip9','ACCOUNT_IDENTIFIER': 'n_acc', 'DELIVERY_SERVICE_NAME': 'service_name'})


df = df[['zip5', 'n_acc', 'service_name', 'date_time', 'energy']]    

MemoryError: 

In [54]:
df


,ZIP_CODE,ACCOUNT_IDENTIFIER,DELIVERY_SERVICE_NAME,Time,energy
0,60002,100165079286762,RESIDENTIAL SINGLE,0030,0.7000
1,60002,1001612044853455,RESIDENTIAL SINGLE,0030,0.1950
2,60002,1001618796973926,RESIDENTIAL SINGLE,0030,0.3575
3,60002,1001621807998827,RESIDENTIAL SINGLE,0030,0.5550
4,60002,1001625165455245,RESIDENTIAL SINGLE,0030,0.0475
...,...,...,...,...,...
127422907,60617,1001706220068127931,MED (100 - 400),0000,60.7680
127422908,60617,1001706491419733703,MED (100 - 400),0000,40.9680
127422909,60617,1001706995463356353,MED (100 - 400),0000,11.8200
127422910,60617,1001707360696072175,MED (100 - 400),0000,41.2490


J:\PhD\Projects\chicago-energy-cons\env\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [51]:
df.memory_usage().sum()

5096916608